<a href="https://colab.research.google.com/github/Rocking-Priya/704-fall-projects-2025/blob/main/Week_4_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DX 704 Week 4 Project
This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [26]:
import numpy as np
import pandas as pd

np.random.seed(42)  # make sampling reproducible

# --- 1) Load the tag file and make one row per recipe_slug ---
tags = pd.read_csv('/content/recipe-tags.tsv', sep="\t")   # has many rows per recipe
recipes = tags.groupby('recipe_slug', as_index=False).agg({
    'recipe_tag': lambda s: " ".join(sorted(set([t.strip() for t in s.dropna()])))
})
# now recipes has 1 row per recipe_slug and a string of tags per recipe

print("Unique recipes:", len(recipes))  # should be 100 for the assignment

# --- 2) Create boolean columns for the key tags we care about (persona) ---
recipes['has_chocolate'] = recipes['recipe_tag'].str.contains('chocolate', case=False, na=False)
recipes['has_vegetarian'] = recipes['recipe_tag'].str.contains('vegetarian', case=False, na=False)
recipes['has_bacon'] = recipes['recipe_tag'].str.contains('bacon', case=False, na=False)

# --- 3) Start with a raw-rating (1..5) default neutral=3 ---
recipes['rating_raw'] = 3

# Persona rules:
# chocolate lover -> best = 5
recipes.loc[recipes['has_chocolate'], 'rating_raw'] = 5

# vegetarian -> good = 4 (but chocolate takes precedence if both True)
recipes.loc[(recipes['has_vegetarian']) & (~recipes['has_chocolate']), 'rating_raw'] = 4

# bacon -> bad = 1 (lowest)
recipes.loc[recipes['has_bacon'], 'rating_raw'] = 1

# --- 4) Make sure we have at least 10 zeros (rating_raw == 1) and 10 ones (rating_raw == 5)
# Count current extremes
count_zeros = (recipes['rating_raw'] == 1).sum()
count_ones  = (recipes['rating_raw'] == 5).sum()

# If not enough zeros, randomly set some other recipes to 1
if count_zeros < 10:
    needed = 10 - count_zeros
    candidates = recipes[recipes['rating_raw'] != 1].sample(needed, random_state=42).index
    recipes.loc[candidates, 'rating_raw'] = 1

# If not enough ones, randomly set some other recipes to 5
if count_ones < 10:
    needed = 10 - count_ones
    candidates = recipes[recipes['rating_raw'] != 5].sample(needed, random_state=43).index
    recipes.loc[candidates, 'rating_raw'] = 5

# Recount and show
count_zeros = (recipes['rating_raw'] == 1).sum()
count_ones  = (recipes['rating_raw'] == 5).sum()
print("Final counts -> raw==1 (will map to 0):", count_zeros, ", raw==5 (will map to 1):", count_ones)

# --- 5) Map raw ratings 1..5 to 0..1 using the provided formula ---
recipes['rating'] = (recipes['rating_raw'] - 1) * 0.25

# --- 6) Keep only required columns and save ---
ratings_df = recipes[['recipe_slug', 'rating']]
ratings_df.to_csv("ratings.tsv", sep="\t", index=False)
print("Saved ratings.tsv (first 10 rows):")
print(ratings_df.head(10))


Unique recipes: 100
Final counts -> raw==1 (will map to 0): 15 , raw==5 (will map to 1): 10
Saved ratings.tsv (first 10 rows):
                        recipe_slug  rating
0               almond-chip-cookies    1.00
1                 almond-croissants    0.50
2                       apple-crisp    0.50
3                     apple-crumble    0.50
4                         apple-pie    0.50
5                  asparagus-burger    0.75
6                  asparagus-quiche    0.75
7  bacon-and-egg-breakfast-sandwich    0.00
8      bacon-chocolate-chip-cookies    0.00
9                 bacon-egg-muffins    0.00


In [27]:
r = pd.read_csv("ratings.tsv", sep="\t")
print(r['rating'].value_counts())
print((r['rating'] == 0.0).sum(), "zeros; ", (r['rating'] == 1.0).sum(), "ones")


rating
0.50    58
0.75    17
0.00    15
1.00    10
Name: count, dtype: int64
15 zeros;  10 ones


In [28]:
from google.colab import files

files.download("ratings.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [29]:


# 1) Load files (once)
recipes = pd.read_csv('/content/recipe-tags.tsv', sep="\t")   # long format: many rows per recipe
ratings = pd.read_csv("ratings.tsv", sep="\t")               # one row per recipe, desired ordering

# 2) Clean tags: strip whitespace and lower-case to make consistent tag names
recipes['recipe_tag'] = recipes['recipe_tag'].astype(str).str.strip().str.lower()

# 3) One-hot encode tags
#    prefix='' removes the 'recipe_tag_' prefix so columns are the raw tag names
#    prefix_sep='' avoids adding a separator
dummies = pd.get_dummies(recipes['recipe_tag'], prefix='', prefix_sep='')

# 4) Put back the slug so we can groupby
dummies['recipe_slug'] = recipes['recipe_slug']

# 5) Aggregate to one row per recipe (take max so any tag present becomes 1)
features_per_recipe = dummies.groupby('recipe_slug').max().reset_index()

# 6) Ensure numeric 0/1 (in case groupby yielded booleans)
for col in features_per_recipe.columns:
    if col != 'recipe_slug':
        features_per_recipe[col] = pd.to_numeric(features_per_recipe[col], errors='coerce').fillna(0).astype(int)

# 7) Add bias column (hard-coded 1)
features_per_recipe['bias'] = 1

# 8) Reorder rows to match ratings.csv (important)
#    This keeps features.tsv and ratings.tsv aligned row-by-row.
features_per_recipe = features_per_recipe.set_index('recipe_slug').reindex(ratings['recipe_slug']).reset_index()

# 9) Optional: put recipe_slug and bias first, then tags (a tidy layout)
cols = ['recipe_slug', 'bias'] + [c for c in features_per_recipe.columns if c not in ('recipe_slug','bias')]
features_per_recipe = features_per_recipe[cols]

# 10) Save TSV without extra index column
features_per_recipe.to_csv("features.tsv", sep="\t", index=False)

print("Created features.tsv with shape:", features_per_recipe.shape)
print(features_per_recipe.head())


Created features.tsv with shape: (100, 298)
           recipe_slug  bias  alfredo  almond  american  appetizer  \
0  almond-chip-cookies     1        0       1         0          0   
1    almond-croissants     1        0       1         0          0   
2          apple-crisp     1        0       0         0          0   
3        apple-crumble     1        0       0         0          0   
4            apple-pie     1        0       0         0          0   

   appetizers  apple  asiancuisine  asparagus  ...  udonnoodles  vanilla  \
0           0      0             0          0  ...            0        0   
1           0      0             0          0  ...            0        0   
2           0      1             0          0  ...            0        0   
3           0      1             0          0  ...            0        0   
4           0      1             0          0  ...            0        0   

   vanillaicecream  vegan  vegetables  vegetarian  warm  whippedcream  winter 

/tmp/ipython-input-1138562594.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_per_recipe['bias'] = 1


In [30]:
# quick checks
f = pd.read_csv("features.tsv", sep="\t")
r = pd.read_csv("ratings.tsv", sep="\t")
print("Shapes:", f.shape, r.shape)
print("First columns:", list(f.columns[:6]))
print("Bias unique:", f['bias'].unique())
print("Numeric dtypes?", all(dt.kind in 'biuf' for dt in f.drop('recipe_slug', axis=1).dtypes))
# Ensure same slugs and order:
print("Slugs align exactly?", (f['recipe_slug'].tolist() == r['recipe_slug'].tolist()))


Shapes: (100, 298) (100, 2)
First columns: ['recipe_slug', 'bias', 'alfredo', 'almond', 'american', 'appetizer']
Bias unique: [1]
Numeric dtypes? True
Slugs align exactly? True


In [31]:
files.download("features.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [32]:

from sklearn.linear_model import Ridge

# Load (once)
features_df = pd.read_csv("features.tsv", sep="\t")
ratings_df  = pd.read_csv("ratings.tsv", sep="\t")

# Reorder features to match ratings (one operation)
features_df = features_df.set_index('recipe_slug').reindex(ratings_df['recipe_slug']).reset_index()

# Prepare X and y
X = features_df.drop(columns=['recipe_slug'])
X = X.apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)  # ensure numeric
y = ratings_df['rating'].astype(float)

# Train Ridge with alpha=1 and no intercept (bias column used instead)
model = Ridge(alpha=1, fit_intercept=False)
model.fit(X, y)

print("Ridge regression model trained successfully.")


Ridge regression model trained successfully.


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [33]:
# YOUR CHANGES HERE

# Save the coefficients to model.tsv
coefficients_df = pd.DataFrame({
    'recipe_tag': X.columns,       # column names = tags + bias
    'coefficient': model.coef_     # learned weights
})

# Do NOT add intercept_ (already covered by bias column)
coefficients_df.to_csv("model.tsv", sep="\t", index=False)

print("Model coefficients saved to model.tsv")
print(coefficients_df.head())


Model coefficients saved to model.tsv
  recipe_tag  coefficient
0       bias     0.387317
1    alfredo     0.005582
2     almond     0.100951
3   american    -0.001226
4  appetizer     0.007957


In [34]:
# load to verify
m = pd.read_csv("model.tsv", sep="\t")
print("rows in model.tsv:", len(m))
print("bias coefficient:", m.loc[m['recipe_tag']=='bias', 'coefficient'].values)
# check alignment
print("X columns count:", X.shape[1], "coeff count:", len(model.coef_))

rows in model.tsv: 297
bias coefficient: [0.38731716]
X columns count: 297 coeff count: 297


In [35]:
files.download("model.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [36]:
# YOUR CHANGES HERE
# 1) Load features and coefficients
features_df = pd.read_csv("features.tsv", sep="\t")   # Part 2 output
coeff_df    = pd.read_csv("model.tsv", sep="\t")      # Part 3 output

# 2) Prepare feature matrix X (use recipe_slug as index)
X = features_df.set_index('recipe_slug')

# 3) Ensure feature columns are numeric (coerce True/False -> 1/0; strings -> 0)
X = X.apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)

# 4) Build coefficient Series (index = feature name)
coef = pd.Series(coeff_df['coefficient'].values, index=coeff_df['recipe_tag'].values)

# 5) Align coefficients to X.columns (fill missing features with zero)
coef = coef.reindex(X.columns).fillna(0).astype(float)

# 6) Compute estimates: dot product X · coef
score_series = X.dot(coef)   # pandas Series indexed by recipe_slug

# 7) Build DataFrame and save
estimates_df = score_series.reset_index()
estimates_df.columns = ['recipe_slug', 'score_estimate']

estimates_df.to_csv("estimates.tsv", sep="\t", index=False)
print("Created estimates.tsv with", len(estimates_df), "rows")
print(estimates_df.head())

Created estimates.tsv with 100 rows
           recipe_slug  score_estimate
0  almond-chip-cookies        0.853245
1    almond-croissants        0.538100
2          apple-crisp        0.505219
3        apple-crumble        0.490581
4            apple-pie        0.522238


In [37]:

files.download("estimates.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [38]:
import numpy as np
import pandas as pd

# Load data
features_df = pd.read_csv("features.tsv", sep="\t")
ratings_df  = pd.read_csv("ratings.tsv", sep="\t")

# Align feature rows to ratings order
features_df = features_df.set_index('recipe_slug').reindex(ratings_df['recipe_slug']).reset_index()

# Numeric feature matrix D and reward vector c
X = features_df.drop(columns=['recipe_slug']).apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)
D = X.to_numpy()                           # (n_recipes, n_features)
c = ratings_df['rating'].to_numpy()        # (n_recipes,)

# LinUCB parameter
alpha = 2.0

# Compute (I + D^T D) and inverse (ridge lambda = 1)
DTD_plus_I = np.eye(D.shape[1]) + D.T @ D
DTD_plus_I_inv = np.linalg.inv(DTD_plus_I)

# theta_hat (ridge closed form)
theta_hat = DTD_plus_I_inv @ (D.T @ c)

# predicted means and variances
means = D @ theta_hat
variances = np.array([ (z.reshape(-1,1).T @ DTD_plus_I_inv @ z.reshape(-1,1)).item() for z in D ])

# LinUCB bound
score_bound = means + alpha * np.sqrt(variances)

# Save
bounds_df = pd.DataFrame({"recipe_slug": features_df['recipe_slug'], "score_bound": score_bound})
bounds_df.to_csv("bounds.tsv", sep="\t", index=False)


In [39]:
files.download("bounds.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [40]:
# YOUR CHANGES HERE



# -------------------------
# 0. Parameters
# -------------------------
alpha = 2.0        # LinUCB exploration parameter (given in Part 5)
# lambda is handled by initializing A = I (equivalent to regularizer = 1)

# -------------------------
# 1. Load features and ratings
# -------------------------
features_df = pd.read_csv("features.tsv", sep="\t")   # from Part 2
ratings_df  = pd.read_csv("ratings.tsv", sep="\t")    # from Part 1

# Make sure features and ratings refer to the same recipes in the same order
# (this is convenient but not strictly required for correctness)
features_df = features_df.set_index('recipe_slug').reindex(ratings_df['recipe_slug']).fillna(0)
ratings_series = ratings_df.set_index('recipe_slug')['rating']

# Convert feature table to numeric matrix X (rows = recipes, cols = features including 'bias')
X_df = features_df.astype(float)
recipe_slugs = X_df.index.to_list()
X = X_df.to_numpy(dtype=float)   # shape (n_recipes, n_features)
n, p = X.shape

# -------------------------
# 2. Initialize LinUCB variables
# -------------------------
A = np.eye(p, dtype=float)     # A = I (p x p). This encodes the ridge regularizer = 1.
b = np.zeros(p, dtype=float)   # b = 0 (p-vector)

# We'll record the recommendations in order
records = []  # list of tuples (recipe_slug, score_bound_at_choice, reward_observed)

# -------------------------
# 3. Online loop: make 100 recommendations
# -------------------------
T = 100
for t in range(T):
    # 3.1 compute current inverse and theta_hat
    A_inv = np.linalg.inv(A)        # inverse of A
    theta_hat = A_inv @ b           # current linear estimate (p-vector)

    # 3.2 compute mean estimates and variances for every recipe
    means = X @ theta_hat           # shape (n,) predicted mean for each recipe
    # variance for each recipe: x^T A_inv x
    variances = np.array([float(x @ A_inv @ x) for x in X])   # shape (n,)

    # 3.3 compute LinUCB upper bound = mean + alpha * stddev
    bounds = means + alpha * np.sqrt(variances)

    # 3.4 pick the recipe with the largest bound (ties broken by first occurrence)
    j = int(np.argmax(bounds))
    chosen_slug = recipe_slugs[j]
    chosen_bound = float(bounds[j])

    # 3.5 observe reward: use the rating from ratings.tsv (the assignment says to use these)
    reward = float(ratings_series.loc[chosen_slug])

    # 3.6 record the recommendation
    records.append((chosen_slug, chosen_bound, reward))

    # 3.7 update A and b with the chosen context and observed reward
    x = X[j]   # feature vector (1D)
    A += np.outer(x, x)   # A = A + x x^T
    b += reward * x       # b = b + r * x

# -------------------------
# 4. Save recommendations.tsv
# -------------------------
recs_df = pd.DataFrame(records, columns=['recipe_slug', 'score_bound', 'reward'])
recs_df.to_csv("recommendations.tsv", sep="\t", index=False)
print("Saved recommendations.tsv with", len(recs_df), "rows")
print(recs_df.head())


Saved recommendations.tsv with 100 rows
        recipe_slug  score_bound  reward
0     apple-crumble     7.483315     0.5
1     ma-la-chicken     7.225922     0.5
2       quesadillas     7.215690     0.5
3             ramen     7.205736     0.5
4  pain-au-chocolat     6.948626     1.0


In [41]:
files.download("recommendations.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
